In [7]:
from serlo import SerloBot, Environment

bot = SerloBot(Environment.STAGING)
bot.login()

Serlo-Username: ········
Serlo-Password: ········


In [8]:
import json
import mysql.connector

from serlo.mysql import query
from IPython.display import display, Markdown, HTML

def getAllEntityRevisions():
    results = query("""
        select
            max(entity_revision.id),
            max(entity.id),
            max(type.name),
            json_objectagg(entity_revision_field.field, entity_revision_field.value) as fields
        from entity_revision
            join entity on entity_revision.repository_id = entity.id
            join type on entity.type_id = type.id
            join entity_revision_field on entity_revision_field.entity_revision_id = entity_revision.id
        group by entity_revision.id
    """)
    
    return [{
        "revisionId": result[0],
        "entityId": result[1],
        "repositoryId": result[1],
        "type": result[2],
        **json.loads(result[3])
    } for result in results]

getAllEntityRevisions()[0]

{'revisionId': 1496,
 'entityId': 1495,
 'repositoryId': 1495,
 'type': 'article',
 'title': 'Addition',
 'content': '[[{"col":24,"content":"Die **Addition**, umgangssprachlich auch **Plus-Rechnen** genannt, ist eine der vier  [Grundrechenarten](/1871) . In der Grundschule und in der Umgangssprache verwendet man meist den Ausdruck **Zusammenz\\u00e4hlen** f\\u00fcr die Addition von zwei oder mehr Zahlen, da Addition den Vorgang des Z\\u00e4hlens beschreibt.\\n\\n\\u00a0\\n\\nDie Elemente bzw. Operanden einer Addition werden\\u00a0**Summanden**\\u00a0und das Ergebnis\\u00a0**Summe**\\u00a0genannt:\\u00a0\\n\\n1. Summand + 2. Summand = Summe\\n\\nEine Summe muss aber nicht nur aus 2 Summanden bestehen, sie kann auch aus mehreren Summanden bestehen.\\u00a0\\n\\n\\u00a0\\n\\nIm Allgemeinen ist die Addition nicht f\\u00fcr f\\u00fcr Zahlen definiert. Vektoren kann man zum Beispiel auch miteinander addieren. Die Grundrechenart, die eine Addition \\"r\\u00fcckg\\u00e4ngig\\" macht, ist\\u00a0

In [9]:
def getAllPageRevisions():
    results = query("""
        select
            id, page_repository_id, title, content, date, trashed
        from page_revision
    """)
    
    return [{
        "revisionId": result[0],
        "repositoryId": result[1],
        "title": result[2],
        "content": result[3],
        "date": result[4],
        "trashed": result[5],
        "type": "page"
    } for result in results]

getAllPageRevisions()[0]

{'revisionId': 16258,
 'repositoryId': 16256,
 'title': 'Das Team',
 'content': '[[{"col":24,"content":"Einen maÃ\x9fgeblichen Beitrag zu Serlo leisten die aktiven Nutzer, die sich mit der Ausarbeitung und Verbesserung  von LÃ¶sungen und Artikeln beschÃ¤ftigen. Jeder, der aktiv an Serlo arbeitet, kann Mitglied in dem Verein werden. Serlo ist Gemeingut und gehÃ¶rt keinem festen Personenkreis.\\n\\nOrganisation\\n"}],[{"col":6,"content":"![Simon KÃ¶hl](/attachment/file/16257/5)\\n>[zum Profil](10)"},{"col":6,"content":""},{"col":6,"content":""},{"col":6,"content":""}]]',
 'date': datetime.datetime(2014, 3, 7, 0, 28, 9),
 'trashed': 0,
 'type': 'page'}

In [10]:
def parseEdtrState(content):
    try:
        state = json.loads(content)
    except:
        return None
    
    if "plugin" not in state:
        return None
    
    return state

def stateHasPlugin(plugin, state):
    if isinstance(state, dict):
        if "plugin" in state and state["plugin"] == plugin:
            return True
        
        return any(stateHasPlugin(plugin, value) for value in state.values())
    elif isinstance(state, list):
        return any(stateHasPlugin(plugin, value) for value in state)
    else:
        return False

def revisionHasPlugin(plugin, revision):
    if "content" not in revision:
        return False
    
    state = parseEdtrState(revision["content"])
    
    return stateHasPlugin(plugin, state) if state != None else False

In [11]:
def uuid_key(uuid):
    return "de.serlo.org/api/uuid/%s" % uuid

revisions = getAllPageRevisions() + getAllEntityRevisions()
affected_revisions = (r for r in revisions if revisionHasPlugin("important", r) or revisionHasPlugin("blockquote", r))
affected_ids = [106081, 41107] + [r["revisionId"] for r in affected_revisions]
affected_keys = ["serlo.org/subjects"] + [uuid_key(i) for i in affected_ids]

len(affected_keys)

2990

In [12]:
def update_cache(ids, chunk_size=100):
    chunks = [ids[i:i+chunk_size] for i in range(0, len(ids), chunk_size)]
    
    for uuid in ids:
        result = bot.api_call("""
            mutation($input: CacheRemoveInput!) {
                _cache {
                    remove(input: $input) {
                        success
                    }
                }
            }
        """, variables= {
            "input": {
                "key": uuid
            }
        })
        
        print("Remove uuid %s" % uuid)
    
    for chunk in chunks:
        result = bot.api_call("""
            mutation($input: CacheUpdateInput!) {
                _cache {
                    update(input: $input) {
                        success
                    }
                }
            }
        """, variables= {
            "input": {
                "keys": chunk
            }
        })
        
        print(result)

update_cache(affected_keys)

Remove uuid serlo.org/subjects
Remove uuid de.serlo.org/api/uuid/106081
Remove uuid de.serlo.org/api/uuid/41107
Remove uuid de.serlo.org/api/uuid/122986
Remove uuid de.serlo.org/api/uuid/122987
Remove uuid de.serlo.org/api/uuid/129846
Remove uuid de.serlo.org/api/uuid/129847
Remove uuid de.serlo.org/api/uuid/141546
Remove uuid de.serlo.org/api/uuid/141547
Remove uuid de.serlo.org/api/uuid/141549
Remove uuid de.serlo.org/api/uuid/141550
Remove uuid de.serlo.org/api/uuid/141551
Remove uuid de.serlo.org/api/uuid/141552
Remove uuid de.serlo.org/api/uuid/148163
Remove uuid de.serlo.org/api/uuid/148425
Remove uuid de.serlo.org/api/uuid/148812
Remove uuid de.serlo.org/api/uuid/150567
Remove uuid de.serlo.org/api/uuid/167424
Remove uuid de.serlo.org/api/uuid/167425
Remove uuid de.serlo.org/api/uuid/168396
Remove uuid de.serlo.org/api/uuid/169470
Remove uuid de.serlo.org/api/uuid/169697
Remove uuid de.serlo.org/api/uuid/170430
Remove uuid de.serlo.org/api/uuid/173153
Remove uuid de.serlo.org/ap

Remove uuid de.serlo.org/api/uuid/148419
Remove uuid de.serlo.org/api/uuid/148420
Remove uuid de.serlo.org/api/uuid/148441
Remove uuid de.serlo.org/api/uuid/148447
Remove uuid de.serlo.org/api/uuid/148450
Remove uuid de.serlo.org/api/uuid/148453
Remove uuid de.serlo.org/api/uuid/148456
Remove uuid de.serlo.org/api/uuid/148459
Remove uuid de.serlo.org/api/uuid/148462
Remove uuid de.serlo.org/api/uuid/148464
Remove uuid de.serlo.org/api/uuid/148471
Remove uuid de.serlo.org/api/uuid/148609
Remove uuid de.serlo.org/api/uuid/148610
Remove uuid de.serlo.org/api/uuid/148626
Remove uuid de.serlo.org/api/uuid/148650
Remove uuid de.serlo.org/api/uuid/148744
Remove uuid de.serlo.org/api/uuid/148745
Remove uuid de.serlo.org/api/uuid/148754
Remove uuid de.serlo.org/api/uuid/148755
Remove uuid de.serlo.org/api/uuid/148756
Remove uuid de.serlo.org/api/uuid/148757
Remove uuid de.serlo.org/api/uuid/148758
Remove uuid de.serlo.org/api/uuid/148761
Remove uuid de.serlo.org/api/uuid/148762
Remove uuid de.s

Remove uuid de.serlo.org/api/uuid/154939
Remove uuid de.serlo.org/api/uuid/154949
Remove uuid de.serlo.org/api/uuid/154951
Remove uuid de.serlo.org/api/uuid/154991
Remove uuid de.serlo.org/api/uuid/155000
Remove uuid de.serlo.org/api/uuid/155013
Remove uuid de.serlo.org/api/uuid/155019
Remove uuid de.serlo.org/api/uuid/155308
Remove uuid de.serlo.org/api/uuid/155325
Remove uuid de.serlo.org/api/uuid/155337
Remove uuid de.serlo.org/api/uuid/155338
Remove uuid de.serlo.org/api/uuid/155339
Remove uuid de.serlo.org/api/uuid/155341
Remove uuid de.serlo.org/api/uuid/155344
Remove uuid de.serlo.org/api/uuid/155356
Remove uuid de.serlo.org/api/uuid/155362
Remove uuid de.serlo.org/api/uuid/155615
Remove uuid de.serlo.org/api/uuid/155707
Remove uuid de.serlo.org/api/uuid/155807
Remove uuid de.serlo.org/api/uuid/155819
Remove uuid de.serlo.org/api/uuid/155822
Remove uuid de.serlo.org/api/uuid/155828
Remove uuid de.serlo.org/api/uuid/155993
Remove uuid de.serlo.org/api/uuid/156022
Remove uuid de.s

Remove uuid de.serlo.org/api/uuid/163670
Remove uuid de.serlo.org/api/uuid/163685
Remove uuid de.serlo.org/api/uuid/163695
Remove uuid de.serlo.org/api/uuid/163697
Remove uuid de.serlo.org/api/uuid/163702
Remove uuid de.serlo.org/api/uuid/163783
Remove uuid de.serlo.org/api/uuid/163784
Remove uuid de.serlo.org/api/uuid/163785
Remove uuid de.serlo.org/api/uuid/163792
Remove uuid de.serlo.org/api/uuid/163795
Remove uuid de.serlo.org/api/uuid/163798
Remove uuid de.serlo.org/api/uuid/163799
Remove uuid de.serlo.org/api/uuid/163800
Remove uuid de.serlo.org/api/uuid/164139
Remove uuid de.serlo.org/api/uuid/164220
Remove uuid de.serlo.org/api/uuid/164224
Remove uuid de.serlo.org/api/uuid/164225
Remove uuid de.serlo.org/api/uuid/164607
Remove uuid de.serlo.org/api/uuid/164852
Remove uuid de.serlo.org/api/uuid/164862
Remove uuid de.serlo.org/api/uuid/164863
Remove uuid de.serlo.org/api/uuid/164902
Remove uuid de.serlo.org/api/uuid/164904
Remove uuid de.serlo.org/api/uuid/164905
Remove uuid de.s

Remove uuid de.serlo.org/api/uuid/169538
Remove uuid de.serlo.org/api/uuid/169628
Remove uuid de.serlo.org/api/uuid/169641
Remove uuid de.serlo.org/api/uuid/169647
Remove uuid de.serlo.org/api/uuid/169648
Remove uuid de.serlo.org/api/uuid/169650
Remove uuid de.serlo.org/api/uuid/169706
Remove uuid de.serlo.org/api/uuid/169711
Remove uuid de.serlo.org/api/uuid/169717
Remove uuid de.serlo.org/api/uuid/169718
Remove uuid de.serlo.org/api/uuid/169733
Remove uuid de.serlo.org/api/uuid/169734
Remove uuid de.serlo.org/api/uuid/169735
Remove uuid de.serlo.org/api/uuid/169737
Remove uuid de.serlo.org/api/uuid/169738
Remove uuid de.serlo.org/api/uuid/169740
Remove uuid de.serlo.org/api/uuid/169781
Remove uuid de.serlo.org/api/uuid/169809
Remove uuid de.serlo.org/api/uuid/169814
Remove uuid de.serlo.org/api/uuid/169936
Remove uuid de.serlo.org/api/uuid/169989
Remove uuid de.serlo.org/api/uuid/170033
Remove uuid de.serlo.org/api/uuid/170088
Remove uuid de.serlo.org/api/uuid/170271
Remove uuid de.s

...

Remove uuid de.serlo.org/api/uuid/228618
Remove uuid de.serlo.org/api/uuid/228662
Remove uuid de.serlo.org/api/uuid/228722
Remove uuid de.serlo.org/api/uuid/228792
Remove uuid de.serlo.org/api/uuid/228807
Remove uuid de.serlo.org/api/uuid/228821
Remove uuid de.serlo.org/api/uuid/228822
Remove uuid de.serlo.org/api/uuid/228823
Remove uuid de.serlo.org/api/uuid/228825
Remove uuid de.serlo.org/api/uuid/228827
Remove uuid de.serlo.org/api/uuid/228830
Remove uuid de.serlo.org/api/uuid/228842
Remove uuid de.serlo.org/api/uuid/228844
Remove uuid de.serlo.org/api/uuid/228848
Remove uuid de.serlo.org/api/uuid/228849
Remove uuid de.serlo.org/api/uuid/228861
Remove uuid de.serlo.org/api/uuid/228882
Remove uuid de.serlo.org/api/uuid/228883
Remove uuid de.serlo.org/api/uuid/228932
Remove uuid de.serlo.org/api/uuid/228993
Remove uuid de.serlo.org/api/uuid/229056
Remove uuid de.serlo.org/api/uuid/229162
Remove uuid de.serlo.org/api/uuid/229278
Remove uuid de.serlo.org/api/uuid/229285
Remove uuid de.s

Remove uuid de.serlo.org/api/uuid/234162
Remove uuid de.serlo.org/api/uuid/234167
Remove uuid de.serlo.org/api/uuid/234230
Remove uuid de.serlo.org/api/uuid/234282
Remove uuid de.serlo.org/api/uuid/234328
Remove uuid de.serlo.org/api/uuid/234373
Remove uuid de.serlo.org/api/uuid/234374
Remove uuid de.serlo.org/api/uuid/234375
Remove uuid de.serlo.org/api/uuid/234376
Remove uuid de.serlo.org/api/uuid/234377
Remove uuid de.serlo.org/api/uuid/234378
Remove uuid de.serlo.org/api/uuid/234379
Remove uuid de.serlo.org/api/uuid/234380
Remove uuid de.serlo.org/api/uuid/234381
Remove uuid de.serlo.org/api/uuid/234383
Remove uuid de.serlo.org/api/uuid/234385
Remove uuid de.serlo.org/api/uuid/234386
Remove uuid de.serlo.org/api/uuid/234400
Remove uuid de.serlo.org/api/uuid/234475
Remove uuid de.serlo.org/api/uuid/234480
Remove uuid de.serlo.org/api/uuid/234561
Remove uuid de.serlo.org/api/uuid/234562
Remove uuid de.serlo.org/api/uuid/234564
Remove uuid de.serlo.org/api/uuid/234569
Remove uuid de.s

Remove uuid de.serlo.org/api/uuid/239609
Remove uuid de.serlo.org/api/uuid/239621
Remove uuid de.serlo.org/api/uuid/239622
Remove uuid de.serlo.org/api/uuid/239623
Remove uuid de.serlo.org/api/uuid/239624
Remove uuid de.serlo.org/api/uuid/239639
Remove uuid de.serlo.org/api/uuid/239640
Remove uuid de.serlo.org/api/uuid/239657
Remove uuid de.serlo.org/api/uuid/239679
Remove uuid de.serlo.org/api/uuid/239685
Remove uuid de.serlo.org/api/uuid/239687
Remove uuid de.serlo.org/api/uuid/239705
Remove uuid de.serlo.org/api/uuid/239724
Remove uuid de.serlo.org/api/uuid/239726
Remove uuid de.serlo.org/api/uuid/239730
Remove uuid de.serlo.org/api/uuid/239731
Remove uuid de.serlo.org/api/uuid/239732
Remove uuid de.serlo.org/api/uuid/239749
Remove uuid de.serlo.org/api/uuid/239750
Remove uuid de.serlo.org/api/uuid/239756
Remove uuid de.serlo.org/api/uuid/239787
Remove uuid de.serlo.org/api/uuid/239788
Remove uuid de.serlo.org/api/uuid/239793
Remove uuid de.serlo.org/api/uuid/239806
Remove uuid de.s

Remove uuid de.serlo.org/api/uuid/243043
Remove uuid de.serlo.org/api/uuid/243046
Remove uuid de.serlo.org/api/uuid/243048
Remove uuid de.serlo.org/api/uuid/243057
Remove uuid de.serlo.org/api/uuid/243090
Remove uuid de.serlo.org/api/uuid/243097
Remove uuid de.serlo.org/api/uuid/243098
Remove uuid de.serlo.org/api/uuid/243104
Remove uuid de.serlo.org/api/uuid/243106
Remove uuid de.serlo.org/api/uuid/243110
Remove uuid de.serlo.org/api/uuid/243112
Remove uuid de.serlo.org/api/uuid/243185
Remove uuid de.serlo.org/api/uuid/243235
Remove uuid de.serlo.org/api/uuid/243236
Remove uuid de.serlo.org/api/uuid/243243
Remove uuid de.serlo.org/api/uuid/243246
Remove uuid de.serlo.org/api/uuid/243252
Remove uuid de.serlo.org/api/uuid/243276
Remove uuid de.serlo.org/api/uuid/243278
Remove uuid de.serlo.org/api/uuid/243335
Remove uuid de.serlo.org/api/uuid/243496
Remove uuid de.serlo.org/api/uuid/243582
Remove uuid de.serlo.org/api/uuid/243587
Remove uuid de.serlo.org/api/uuid/243598
Remove uuid de.s

{'_cache': {'update': {'success': True}}}
